# SUALBSP-I: Parser and Gurobi Model
This notebook loads `.alb` instances and builds a Gurobi model to minimize the number of stations (Type-I) with sequence-dependent setup times.

Workflow:
1. Parse an instance file.
2. Inspect the parsed data.
3. Build the optimization model.
4. Solve and print the schedule.
5. Plot the station workloads.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Tuple

import gurobipy as gp
from gurobipy import GRB

import matplotlib.pyplot as plt


In [ ]:

@dataclass
class SUALBInstance:
    """Container for a SUALBSP instance."""
    n_tasks: int
    cycle_time: int
    task_times: Dict[int, int]
    precedences: List[Tuple[int, int]]
    setup_forward: Dict[Tuple[int, int], int]
    setup_backward: Dict[Tuple[int, int], int]


def _parse_section(lines: Iterable[str]) -> Dict[str, List[str]]:
    """Parse sections delimited by <section> headers."""
    sections: Dict[str, List[str]] = {}
    current = None
    for raw in lines:
        line = raw.strip()
        if not line:
            continue
        if line.startswith("<") and line.endswith(">"):
            current = line.strip("<>").strip()
            sections[current] = []
            continue
        if current is None:
            continue
        sections[current].append(line)
    return sections


def parse_alb_file(path: Path) -> SUALBInstance:
    """Load a `.alb` file into structured Python data."""

    sections = _parse_section(path.read_text().splitlines())

    n_tasks = int(sections["number of tasks"][0])
    cycle_time = int(sections["cycle time"][0])

    task_times: Dict[int, int] = {}
    for entry in sections["task times"]:
        task, duration = entry.split()
        task_times[int(task)] = int(duration)

    precedences: List[Tuple[int, int]] = []
    for entry in sections["precedence relations"]:
        i, j = entry.split(",")
        precedences.append((int(i), int(j)))

    setup_forward: Dict[Tuple[int, int], int] = {}
    for entry in sections.get("setup times forward", []):
        i, rest = entry.split(",")
        j, t = rest.split(":")
        setup_forward[(int(i), int(j))] = int(t)

    setup_backward: Dict[Tuple[int, int], int] = {}
    for entry in sections.get("setup times backward", []):
        i, rest = entry.split(",")
        j, t = rest.split(":")
        setup_backward[(int(i), int(j))] = int(t)

    return SUALBInstance(
        n_tasks=n_tasks,
        cycle_time=cycle_time,
        task_times=task_times,
        precedences=precedences,
        setup_forward=setup_forward,
        setup_backward=setup_backward,
    )


In [ ]:

# Example: parse an instance (no optimization yet)
instance_path = Path('DataSets/MiniSet.alb')
instance = parse_alb_file(instance_path)
print(f'Tasks: {instance.n_tasks}, Cycle time: {instance.cycle_time}')
print(f'Task times entries: {len(instance.task_times)}')
print(f'Precedence arcs: {len(instance.precedences)}')
print(f'Forward setups: {len(instance.setup_forward)}, Backward setups: {len(instance.setup_backward)}')

# optional: limit the maximum number of stations in the model
max_stations = None


In [ ]:

# Basic sets and parameters
tasks = list(instance.task_times.keys())
n = instance.n_tasks
cycle = instance.cycle_time
ub_stations = max_stations or n
stations = list(range(1, ub_stations + 1))

# Model and decision variables
model = gp.Model("SUALBSP-I")

x = model.addVars(tasks, stations, vtype=GRB.BINARY, name="x")
u = model.addVars(stations, vtype=GRB.BINARY, name="u")
z = model.addVars(tasks, tasks, stations, vtype=GRB.BINARY, name="z")
f = model.addVars(tasks, stations, vtype=GRB.BINARY, name="first")
l = model.addVars(tasks, stations, vtype=GRB.BINARY, name="last")
b = model.addVars(tasks, tasks, stations, vtype=GRB.BINARY, name="backward")
p = model.addVars(tasks, stations, vtype=GRB.INTEGER, lb=1, ub=n, name="pos")

# Assignment: each task exactly once; station is active if it has tasks
model.addConstrs((x.sum(i, "*") == 1 for i in tasks), name="assign_once")
model.addConstrs((x.sum("*", k) <= n * u[k] for k in stations), name="use_station")

# First/last task consistency per station
model.addConstrs((f.sum("*", k) == u[k] for k in stations), name="one_first_per_station")
model.addConstrs((l.sum("*", k) == u[k] for k in stations), name="one_last_per_station")
model.addConstrs((f[i, k] <= x[i, k] for i in tasks for k in stations), name="first_only_if_assigned")
model.addConstrs((l[i, k] <= x[i, k] for i in tasks for k in stations), name="last_only_if_assigned")

# Pairwise ordering on a station
model.addConstrs((z[i, j, k] <= x[i, k] for i in tasks for j in tasks for k in stations), name="order_only_if_assigned_i")
model.addConstrs((z[i, j, k] <= x[j, k] for i in tasks for j in tasks for k in stations), name="order_only_if_assigned_j")
model.addConstrs((z[i, j, k] + z[j, i, k] <= x[i, k] + x[j, k] for i in tasks for j in tasks if i < j for k in stations), name="order_upper")
model.addConstrs((z[i, j, k] + z[j, i, k] >= x[i, k] + x[j, k] - 1 for i in tasks for j in tasks if i < j for k in stations), name="order_lower")

# Position variable bounds for first/last tasks
model.addConstrs((p[i, k] >= 1 * f[i, k] for i in tasks for k in stations), name="first_pos_lb")
model.addConstrs((p[i, k] <= 1 + (n - 1) * (1 - f[i, k]) for i in tasks for k in stations), name="first_pos_ub")
model.addConstrs((p[i, k] >= n * l[i, k] for i in tasks for k in stations), name="last_pos_lb")
model.addConstrs((p[i, k] <= n + (n - 1) * (1 - l[i, k]) for i in tasks for k in stations), name="last_pos_ub")

# MTZ-style ordering to avoid subtours within a station
big_m = n
model.addConstrs((p[i, k] - p[j, k] + big_m * z[i, j, k] <= big_m - 1 + (1 - x[i, k]) * big_m + (1 - x[j, k]) * big_m for i in tasks for j in tasks if i != j for k in stations), name="mtz_order")

# First task precedes all others; last task has no successor
model.addConstrs((z[i, j, k] >= f[i, k] + x[j, k] - 1 for i in tasks for j in tasks if i != j for k in stations), name="first_precedes_all")
model.addConstrs((z[i, j, k] <= 1 - l[j, k] + x[i, k] for i in tasks for j in tasks if i != j for k in stations), name="last_followed_by_none")

# Backward setup pairing (last task of a station to its first task)
model.addConstrs((b[i, j, k] <= l[i, k] for i in tasks for j in tasks for k in stations), name="backward_last")
model.addConstrs((b[i, j, k] <= f[j, k] for i in tasks for j in tasks for k in stations), name="backward_first")
model.addConstrs((b[i, j, k] >= l[i, k] + f[j, k] - 1 for i in tasks for j in tasks for k in stations), name="backward_iff")

# Precedence across stations and within the same station
station_index = {i: gp.quicksum(k * x[i, k] for k in stations) for i in tasks}
model.addConstrs((station_index[i] <= station_index[j] for i, j in instance.precedences), name="precede_station")
model.addConstrs((z[i, j, k] >= x[i, k] + x[j, k] - 1 for i, j in instance.precedences for k in stations), name="precede_order_same_station")

# Cycle time constraint with forward/backward setup times
forward_default = 0
backward_default = 0
for k in stations:
    processing = gp.quicksum(instance.task_times[i] * x[i, k] for i in tasks)
    forward_setups = gp.quicksum(instance.setup_forward.get((i, j), forward_default) * z[i, j, k] for i in tasks for j in tasks if i != j)
    backward_setups = gp.quicksum(instance.setup_backward.get((i, j), backward_default) * b[i, j, k] for i in tasks for j in tasks)
    model.addConstr(processing + forward_setups + backward_setups <= cycle * u[k], name=f"cycle_{k}")

# Objective: minimize the number of active stations
model.setObjective(gp.quicksum(u[k] for k in stations), GRB.MINIMIZE)


In [ ]:
model.update()
model

In [ ]:
model.optimize()

In [ ]:

# Print results with setup times
if model.status == GRB.OPTIMAL:
    print("Objective value (min #stations):", model.objVal)
    print("Cycle Time C:", cycle)
    print("")

    for k in stations:
        tasks_here = [i for i in tasks if x[i, k].X > 0.5]
        if not tasks_here:
            continue

        seq = sorted(tasks_here, key=lambda i: p[i, k].X)

        print(f"=== Station {k} ===")
        t = 0

        for idx, op in enumerate(seq):
            dur = instance.task_times[op]
            start = t
            end = t + dur

            print(f"Task {op:2d}: processing {dur:2d}  "
                  f"(start={start:3d}, end={end:3d})")

            t = end

            if idx < len(seq) - 1:
                nxt = seq[idx + 1]
                s = instance.setup_forward.get((op, nxt), 0)

                if s > 0:
                    print(f"   → setup forward {s:2d} "
                          f"(between {op} → {nxt})")
                    t += s

        # Backward setup (last → first) if used by the model
        sb = 0
        pair = None

        for i in tasks_here:
            for j in tasks_here:
                if b[i, j, k].X > 0.5:
                    sb = instance.setup_backward.get((i, j), 0)
                    pair = (i, j)
                    break
            if pair:
                break

        if sb > 0:
            print(f"   → setup backward {sb:2d} "
                  f"(between {pair[0]} → {pair[1]})")
            t += sb

        print(f"Total time on station {k}: {t}
")

else:
    print("No solution found")


In [ ]:

if model.SolCount == 0:
    print("⚠️ No solution — skipping plot.")
else:
    # 1) Tasks per station (ordered via the p variable)
    ordered_station_tasks = {}

    for k in stations:
        tasks_here = [i for i in tasks if x[i, k].X > 0.5]
        if tasks_here:
            ordered_station_tasks[k] = sorted(tasks_here, key=lambda i: p[i, k].X)

    # 2) Build the plot
    fig, ax = plt.subplots(figsize=(12, 1.2 * len(ordered_station_tasks) + 2))

    y_labels = []
    y_positions = []

    # Enumerate so labels start at 1
    for row, (k, seq) in enumerate(sorted(ordered_station_tasks.items())):
        y_positions.append(row)
        y_labels.append(f"Station {row + 1}")

        # Start time on the station
        t = 0

        for idx, op in enumerate(seq):
            dur = instance.task_times[op]

            # Task bar
            ax.barh(
                row,
                dur,
                left=t,
                label="processing" if (row == 0 and idx == 0) else None
            )

            # Task label
            ax.text(
                t + dur / 2,
                row,
                f"{op}",
                ha="center",
                va="center",
                color="white",
                fontsize=10,
                fontweight="bold"
            )

            t += dur

            # Forward setup
            if idx < len(seq) - 1:
                nxt = seq[idx + 1]
                s = instance.setup_forward.get((op, nxt), 0)

                if s > 0:
                    ax.barh(
                        row,
                        s,
                        left=t,
                        color="gray",
                        alpha=0.6,
                        label="setup (forward)" if (row == 0 and idx == 0) else None
                    )
                    t += s

        # Backward setup (last → first)
        last = seq[-1]
        first = seq[0]
        sb = instance.setup_backward.get((last, first), 0)

        if sb > 0:
            ax.barh(
                row,
                sb,
                left=t,
                color="black",
                alpha=0.55,
                label="setup (backward)" if row == 0 else None
            )
            t += sb

    # Cycle time line
    ax.axvline(cycle, linestyle="--", linewidth=1.5, color="red")
    ax.text(cycle, len(ordered_station_tasks), f"C = {cycle}", color="red")

    # Axes & layout
    ax.set_yticks(y_positions)
    ax.set_yticklabels(y_labels)
    ax.set_xlabel("Time")
    ax.set_title("SUALBSP — Station Workload (Tasks + Setup Times)")
    ax.invert_yaxis()

    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend()

    plt.tight_layout()
    plt.show()
